In [ ]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.cloud import storage

In [ ]:
# Name
filepath = "********"
output_filepath = "********"
text_file_name = "*******.txt"
credentials = "*******.json"
bucket_name = "*******"
source_file_name = filepath + text_file_name
destination_blob_name = text_file_name

In [ ]:
#Uplodaing the data to the cloud
def upload_blob(bucket_name, source_file_name, destination_blob_name, credentials):
    """Uploads a file to the bucket."""
    storage_client = storage.Client.from_service_account_json(credentials)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

upload_blob(bucket_name, source_file_name, destination_blob_name,credentials)

In [ ]:
# 1.Analyzing Sentiment from Google Cloud Storage
gcs_uri = 'gs://*******/' + text_file_name
client = language.LanguageServiceClient.from_service_account_json(credentials)

# Instantiates a plain text document.
document = types.Document(
    gcs_content_uri=gcs_uri,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects sentiment in the document. You can also analyze HTML with:
#   document.type == enums.Document.Type.HTML
sentiment = client.analyze_sentiment(document).document_sentiment

print('Score: {}'.format(sentiment.score))
print('Magnitude: {}'.format(sentiment.magnitude))

In [ ]:
# 2.Analyzing Entities
gcs_uri = 'gs://*******/' + text_file_name
client = language.LanguageServiceClient.from_service_account_json(credentials)

# Instantiates a plain text document.
document = types.Document(
    gcs_content_uri=gcs_uri,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects sentiment in the document. You can also analyze HTML with:
#   document.type == enums.Document.Type.HTML
entities = client.analyze_entities(document).entities

for entity in entities:
    entity_type = enums.Entity.Type(entity.type)
    print('=' * 20)
    print(u'{:<16}: {}'.format('name', entity.name))
    print(u'{:<16}: {}'.format('type', entity_type.name))
    print(u'{:<16}: {}'.format('salience', entity.salience))
    print(u'{:<16}: {}'.format('wikipedia_url',
          entity.metadata.get('wikipedia_url', '-')))
    print(u'{:<16}: {}'.format('mid', entity.metadata.get('mid', '-')))


In [ ]:
# 3.Analyzing Entity Sentiment
# Detect and send native Python encoding to receive correct word offsets.
encoding = enums.EncodingType.UTF32
if sys.maxunicode == 65535:
    encoding = enums.EncodingType.UTF16

result = client.analyze_entity_sentiment(document, encoding)

for entity in result.entities:
    print(u'Name: "{}"'.format(entity.name))
    for mention in entity.mentions:
        print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
        print(u'  Content : {}'.format(mention.text.content))
        print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
        print(u'  Sentiment : {}'.format(mention.sentiment.score))
        print(u'  Type : {}'.format(mention.type))
    print(u'Salience: {}'.format(entity.salience))
    print(u'Sentiment: {}\n'.format(entity.sentiment))